In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [1]:
import json
from google.colab import drive

drive.mount('/content/drive')

def extract_slots(tokens, labels):
    slots = []
    current = {}
    current_key = ""
    buffer = []

    for tok, lab in zip(tokens, labels):
        if lab.startswith("B-"):
            # Si venía acumulando algo, guardar
            if current_key and buffer:
                current[current_key] = " ".join(buffer)
                buffer = []
                current_key = ""

            if lab == "B-VERB":
                if current:
                    slots.append(current)
                current = {"intent": tok}
            elif lab == "B-OBJ":
                current_key = "object"
                buffer = [tok]
            elif lab == "B-LOC":
                current_key = "location"
                buffer = [tok]
            elif lab == "B-FILTER":
                current_key = "filter"
                buffer = [tok]
            elif lab == "B-ATTR":
                current_key = "attribute"
                buffer = [tok]
            elif lab == "B-PERSON":
                # Si ya hay intent, se asume persona es recipient/destinatario
                if "intent" in current and current["intent"] in ["bring", "deliver", "give"]:
                    current["recipient"] = tok
                else:
                    current_key = "person"
                    buffer = [tok]
            elif lab == "B-CONTENT":
                current_key = "content"
                buffer = [tok]

        elif lab.startswith("I-"):
            # Seguir acumulando la frase compuesta
            buffer.append(tok)

    # Guardar último buffer si quedó algo
    if current_key and buffer:
        current[current_key] = " ".join(buffer)

    if current:
        slots.append(current)

    return slots


def main():
    input_file = "/content/drive/My Drive/LLM/dataset.jsonl"  # tu archivo original
    output_file = "/content/drive/My Drive/LLM/dataset_with_slots.jsonl"  # archivo nuevo

    with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
        for line in f_in:
            sample = json.loads(line)
            tokens = sample["tokens"]
            labels = sample["labels"]
            slots = extract_slots(tokens, labels)
            sample["slots"] = slots
            f_out.write(json.dumps(sample) + "\n")

    print(f"✅ Guardado: {output_file}")

if __name__ == "__main__":
    main()


Mounted at /content/drive
✅ Guardado: /content/drive/My Drive/LLM/dataset_with_slots.jsonl


In [3]:
import json
from collections import Counter

# 📌 Ruta a tu archivo JSONL en tu Drive
input_file = "/content/drive/My Drive/LLM/dataset_with_slots.jsonl"

# Contador de intents
intent_counter = Counter()

with open(input_file) as fin:
    for line in fin:
        ex = json.loads(line)
        slots = ex.get("slots", [])
        for s in slots:
            intent = s.get("intent", "UNKNOWN")
            intent_counter[intent] += 1

print("📌 Intents encontrados y frecuencia:")
for intent, count in intent_counter.most_common():
    print(f"{intent}: {count}")


📌 Intents encontrados y frecuencia:
tell: 412
follow: 111
bring: 73
look: 73
meet: 72
find: 71
take: 68
locate: 67
give: 67
get: 61
deliver: 61
answer: 60
navigate: 50
grasp: 49
go: 45
say: 40
fetch: 39
escort: 36
put: 30
lead: 26
place: 25
guide: 23
introduce: 18
greet: 12
salute: 12


In [46]:
import json

POSE_WORDS = [
    "lying", "sitting", "standing", "walking", "waiting",
    "talking", "waving", "pointing", "raising"
]

def fix_slots(slots):
    """Asegura que bring, deliver o give tengan grasp si falta"""
    new_slots = []
    for s in slots:
        if s["intent"] in ["give", "deliver", "bring"]:
            if "object" in s and "location" in s:
                if not any(sl["intent"] == "grasp" for sl in slots):
                    new_slots.append({
                        "intent": "grasp",
                        "object": s["object"],
                        "location": s["location"]
                    })
        new_slots.append(s)
    return new_slots

def slots_to_plan(slots):
    plan_steps = []
    last_location = ""
    last_object = ""
    last_person = ""

    for s in slots:
        intent = s.get("intent", "").lower()
        obj = s.get("object", "")
        loc = s.get("location", "")
        content = s.get("content", "")
        person = s.get("person", "")
        filter_ = s.get("filter", "")
        attr = s.get("attribute", "")
        recipient = s.get("recipient", "")

        if loc: last_location = loc
        if obj: last_object = obj
        if person: last_person = person

        # Si es ir o navegar
        if intent in ["navigate", "go"]:
            plan_steps.append(f"navigate to {loc}")

        # Si hay location en otra acción => navegar primero
        elif loc and intent not in ["navigate", "go"]:
            plan_steps.append(f"navigate to {loc}")

        # === Localizar ===
        if intent in ["locate", "find", "look", "fetch"]:
            if obj:
                step = f"detect_object {obj}"
                if loc:
                    step += f" at {loc}"
                plan_steps.append(step)
            elif person or filter_:
                step = "detect_person"
                if filter_:
                    if filter_ in POSE_WORDS:
                        step += f" with pose {filter_}"
                    else:
                        step += f" with {filter_}"
                if loc:
                    step += f" at {loc}"
                plan_steps.append(step)
            elif attr:
                step = f"detect_object {attr}"
                if loc:
                    step += f" at {loc}"
                step += " -> describe_property"
                plan_steps.append(step)

        # === Agarrar ===
        elif intent in ["get", "grasp", "take"]:
            plan_steps.append("grasp")

        # === Traer o dar ===
        elif intent in ["bring", "deliver", "give"]:
            if not obj:
                obj = last_object
            step = "deliver"
            if obj: step += f" {obj}"
            # Por sentido: navega hasta receptor u operador si hay persona o recipient
            target = person or recipient or "operator"
            step += f" to {target}"
            plan_steps.append(step)

        # === Poner ===
        elif intent in ["put", "place"]:
            step = "place"
            if obj: step += f" {obj}"
            if loc: step += f" at {loc}"
            plan_steps.append(step)

        # === Responder ===
        elif intent == "answer":
            if person or filter_:
                step = "detect_person"
                if filter_ in POSE_WORDS:
                    step += f" with pose {filter_}"
                elif filter_:
                    step += f" with {filter_}"
                if loc:
                    step += f" at {loc}"
                plan_steps.append(step)
            plan_steps.append("navigate to operator -> answer_question")

        # === Presentar ===
        elif intent == "introduce":
            plan_steps.append("introduce_person")

        # === Saludo o decir ===
        elif intent in ["greet", "salute", "say"]:
            step = "greet_person"
            if person:
                step += f" {person}"
            if filter_:
                step += f" with {filter_}"
            if loc:
                step += f" at {loc}"
            plan_steps.append(step)

        # === Guiar, acompañar ===
        elif intent in ["guide", "lead", "escort", "follow"]:
            step = "guide_person"
            if person:
                step += f" {person}"
            if loc:
                step += f" to {loc}"
            plan_steps.append(step)

        # === Encontrar ===
        elif intent == "meet":
            step = "meet_person"
            if person:
                step += f" {person}"
            if loc:
                step += f" at {loc}"
            plan_steps.append(step)

        # === Decir ===
        elif intent == "tell":
            if attr:
                step = f"detect_object {attr}"
                if obj:
                    step += f" {obj}"
                if loc:
                    step += f" at {loc}"
                step += " -> describe_property"
                plan_steps.append(step)
                plan_steps.append("navigate to operator -> tell")
            elif content:
                plan_steps.append(f"tell {content}")
            elif person or filter_:
                step = "detect_person"
                if filter_ in POSE_WORDS:
                    step += f" with pose {filter_}"
                elif filter_:
                    step += f" with {filter_}"
                if loc:
                    step += f" at {loc}"
                step += " -> describe_property"
                plan_steps.append(step)
                plan_steps.append("navigate to operator -> tell")
            elif obj:
                step = f"count {obj}"
                if loc:
                    step += f" at {loc}"
                plan_steps.append(step)
                plan_steps.append("navigate to operator -> tell")

    return " -> ".join(plan_steps)

def main():
    input_file = "/content/drive/My Drive/LLM/dataset_with_slots.jsonl"
    output_file = "/content/drive/My Drive/LLM/dataset_with_slots_and_plan.jsonl"

    with open(input_file) as fin, open(output_file, "w") as fout:
        for line in fin:
            ex = json.loads(line)
            slots_fixed = fix_slots(ex["slots"])
            ex["plan"] = slots_to_plan(slots_fixed)
            fout.write(json.dumps(ex) + "\n")

    print(f"✅ Dataset con planes robustos listo: {output_file}")

if __name__ == "__main__":
    main()


✅ Dataset con planes robustos listo: /content/drive/My Drive/LLM/dataset_with_slots_and_plan.jsonl


In [49]:
import json
import csv

input_file = "/content/drive/My Drive/LLM/dataset_with_slots_and_plan.jsonl"
output_file = "/content/drive/My Drive/LLM/dataset_for_t5.csv"

with open(input_file, "r") as f_in, open(output_file, "w", newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(["input", "output"])  # CSV header

    for line in f_in:
        sample = json.loads(line)
        slots = sample["slots"]
        plan = sample["plan"]

        slot_str = " ".join(
            [f"intent: {s.get('intent', '')}"
             + (f" object: {s['object']}" if 'object' in s else '')
             + (f" location: {s['location']}" if 'location' in s else '')
             + (f" filter: {s['filter']}" if 'filter' in s else '')
             + (f" recipient: {s['recipient']}" if 'recipient' in s else '')
             + (f" attribute: {s['attribute']}" if 'attribute' in s else '')
             + (f" content: {s['content']}" if 'content' in s else '')
             + (f" person: {s['person']}" if 'person' in s else '')
             for s in slots]
        )

        writer.writerow([slot_str.strip(), plan])

print(f"✅ Saved: {output_file}")



✅ Saved: /content/drive/My Drive/LLM/dataset_for_t5.csv


In [50]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

MODEL_NAME = "t5-small"

# Load CSV
df = pd.read_csv("/content/drive/My Drive/LLM/dataset_for_t5.csv")
dataset = Dataset.from_pandas(df)

# Split train/valid
ds = dataset.train_test_split(test_size=0.2)

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def preprocess(example):
    inputs = tokenizer(example["input"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(example["output"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

ds_enc = ds.map(preprocess, batched=True, remove_columns=["input", "output"])

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

args = TrainingArguments(
    output_dir="./t5_plan_model",
    eval_strategy="steps",  # Changed from evaluation_strategy
    eval_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_steps=200,
    logging_steps=20,
    save_total_limit=1,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["test"],
    tokenizer=tokenizer,
)

trainer.train()
model.save_pretrained("/content/drive/My Drive/LLM//t5_plan_model")
tokenizer.save_pretrained("/content/drive/My Drive/LLM//t5_plan_model")

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/tmp/ipython-input-50-2631434905.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
50,2.085200,0.756678
100,0.604600,0.410688
150,0.390400,0.221978
200,0.241800,0.145785
250,0.167400,0.080794
300,0.113500,0.045745
350,0.089100,0.033285
400,0.074200,0.026813
450,0.063900,0.023045
500,0.056100,0.018881


('/content/drive/My Drive/LLM//t5_plan_model/tokenizer_config.json',
 '/content/drive/My Drive/LLM//t5_plan_model/special_tokens_map.json',
 '/content/drive/My Drive/LLM//t5_plan_model/spiece.model',
 '/content/drive/My Drive/LLM//t5_plan_model/added_tokens.json')

In [52]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# 📂 Ruta de tu modelo entrenado:
MODEL_DIR = "/content/drive/My Drive/LLM/t5_plan_model"

# Cargar modelo y tokenizer
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR)
model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

def generate_plan(slot_input):
    input_ids = tokenizer(slot_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128)
    plan = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return plan

# 🧪 Ejemplo de prueba:
example_slots = "intent: locate object: cans location: bedroom intent: get intent: deliver recipient: person"
plan = generate_plan(example_slots)

print("=== INPUT SLOTS ===")
print(example_slots)
print("\n=== GENERATED PLAN ===")
print(plan)


=== INPUT SLOTS ===
intent: locate object: cans location: bedroom intent: get intent: deliver recipient: person

=== GENERATED PLAN ===
navigate to bedroom -> detect_object cans at bedroom -> deliver cans to person


In [34]:
import sys
sys.path.append("/content/drive/My Drive/LLM/CommandGenerator/src")
print(sys.path)


['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmpmocbt8md', '/content/drive/My Drive/LLM/CommandGenerator/src', '/content/drive/My Drive/LLM/CommandGenerator/src']


In [ ]:
import random
import re
import warnings
import os
import torch

from robocupathome_generator.gpsr_commands import CommandGenerator
from robocupathome_generator.egpsr_commands import EgpsrCommandGenerator

from transformers import BertTokenizerFast, BertForTokenClassification, T5Tokenizer, T5ForConditionalGeneration

# === LOAD MODELS ===
BERT_PATH = "/content/drive/My Drive/LLM/bert_token_classification_model/checkpoint-339"
T5_PATH = "/content/drive/My Drive/LLM/t5_plan_model"

bert_tokenizer = BertTokenizerFast.from_pretrained(BERT_PATH)
bert_model = BertForTokenClassification.from_pretrained(BERT_PATH)
bert_model.eval()
id2label = bert_model.config.id2label

t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH)

# === BERT inference ===
def run_inference(command):
    words = command.strip().split()
    if not words:
        print("\n[⚠️] Empty command.")
        return [], []

    inputs = bert_tokenizer(words, is_split_into_words=True, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(**inputs)

    preds = outputs.logits.argmax(dim=-1).squeeze().tolist()
    tokens = bert_tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())

    print("\n=== BERT INFERENCE ===")
    print("TOKEN\tLABEL")
    for t, p in zip(tokens, preds):
        print(f"{t}\t{id2label.get(p, 'O')}")
    print("----------------------")

    labels = [id2label.get(p, "O") for p in preds]
    return tokens, labels

# === Combine subwords ===
def combine_subwords(tokens, labels):
    new_tokens = []
    new_labels = []
    skip = False

    for i in range(len(tokens)):
        if skip:
            skip = False
            continue

        token = tokens[i]
        label = labels[i]

        if token.startswith("##") and new_tokens:
            new_tokens[-1] = new_tokens[-1] + token[2:]
        elif i + 1 < len(tokens) and tokens[i + 1].startswith("##"):
            new_tokens.append(token + tokens[i + 1][2:])
            new_labels.append(label)
            skip = True
        else:
            new_tokens.append(token)
            new_labels.append(label)

    return new_tokens, new_labels

# === Slots extractor ===
def extract_slots(tokens, labels):
    slots = []
    current = {}

    for tok, lab in zip(tokens, labels):
        if lab.startswith("B-VERB"):
            if current:
                slots.append(current)
            current = {"intent": tok}
        elif lab.startswith("B-OBJ"):
            current["object"] = tok
        elif lab.startswith("B-LOC"):
            current["location"] = tok
        elif lab.startswith("I-LOC") and "location" in current:
            current["location"] += " " + tok
        elif lab.startswith("B-ATTR"):
            current["attribute"] = tok
        elif lab.startswith("B-FILTER"):
            current["filter"] = tok
        elif lab.startswith("B-PERSON"):
            current["person"] = tok
        elif lab.startswith("B-CONTENT"):
            current["content"] = tok

    if current:
        slots.append(current)
    return slots

# === Clean slots ===
def clean_slots(slots):
    for s in slots:
        attr = s.get("attribute", "")
        if attr.startswith("##"):
            s["attribute"] = attr.replace("##", "")
        if "filter" in s and "intent" not in s:
            s["intent"] = "locate"
    return slots

def slots_to_string(slots):
    slots = clean_slots(slots)
    return " | ".join(" ".join(f"{k}: {v}" for k, v in s.items()) for s in slots)

# === T5 plan ===
def t5_generate_plan(slots_input):
    enc = t5_tokenizer(slots_input, return_tensors="pt")
    out = t5_model.generate(**enc, max_length=50)
    plan = t5_tokenizer.decode(out[0], skip_special_tokens=True)
    return plan

# === Data utils ===
def read_data(path):
    with open(path) as f:
        return f.read()

def parse_names(data):
    return re.findall(r"\|\s*([A-Za-z]+)\s*\|", data, re.DOTALL)[1:]

def parse_locations(data):
    parsed = re.findall(r"\|\s*([0-9]+)\s*\|\s*([A-Za-z,\s,\(\)]+)\|", data, re.DOTALL)
    raw = [b.strip() for (_, b) in parsed]
    plocs = [b.replace("(p)", "").strip() for b in raw if "(p)" in b]
    locs = [b.replace("(p)", "").strip() for b in raw]
    return locs, plocs

def parse_rooms(data):
    return re.findall(r"\|\s*(\w+ \w*)\s*\|", data, re.DOTALL)[1:]

def parse_objects(data):
    parsed = re.findall(r"\|\s*(\w+)\s*\|", data, re.DOTALL)
    objects = [o.replace("_", " ") for o in parsed if o != "Objectname"]
    cats = re.findall(r"# Class \s*([\w,\s,\(\)]+)\s*", data, re.DOTALL)
    cats = [c.replace("(", "").replace(")", "").split() for c in cats]
    plurals = [c[0].replace("_", " ") for c in cats]
    singulars = [c[1].replace("_", " ") for c in cats]
    return objects, plurals, singulars

# === Mount your paths ===
data_dir = "/content/drive/My Drive/LLM/CommandGenerator"
names = parse_names(read_data(f"{data_dir}/names/names.md"))
locs, plocs = parse_locations(read_data(f"{data_dir}/maps/location_names.md"))
rooms = parse_rooms(read_data(f"{data_dir}/maps/room_names.md"))
objs, cats_plural, cats_singular = parse_objects(read_data(f"{data_dir}/objects/objects.md"))

gpsr_gen = CommandGenerator(names, locs, plocs, rooms, objs, cats_plural, cats_singular)
egpsr_gen = EgpsrCommandGenerator(gpsr_gen)

# === MAIN ===
print("=== OPTIONS ===")
print("'1': Any GPSR command")
print("'2': GPSR no manipulation")
print("'3': GPSR manipulation")
print("'4': EGPSR multi-task (5 tasks)")
print("'q': Quit\n")

while True:
    option = input("Select option: ").strip()
    if option == "q":
        break

    if option == "1":
        command = gpsr_gen.generate_command_start("")
    elif option == "2":
        command = gpsr_gen.generate_command_start("people")
    elif option == "3":
        command = gpsr_gen.generate_command_start("objects")
    elif option == "4":
        setups = egpsr_gen.generate_setup(5)
        command = "\n".join([f"{i+1}) {t.task}" for i, t in enumerate(setups)])
    else:
        print("[⚠️] Invalid option")
        continue

    print(f"\n=== COMMAND ===\n{command}\n")

    tokens, labels = run_inference(command)
    tokens, labels = combine_subwords(tokens, labels)
    slots = extract_slots(tokens, labels)
    slots_str = slots_to_string(slots)
    print("\n=== SLOTS INPUT FOR T5 ===")
    print(slots_str)

    plan = t5_generate_plan(slots_str)
    print("\n=== PLAN GENERATED ===")
    print(plan)
    print("------------------------------")


=== OPTIONS ===
'1': Any GPSR command
'2': GPSR no manipulation
'3': GPSR manipulation
'4': EGPSR multi-task (5 tasks)
'q': Quit

Select option: 1

=== COMMAND ===
go to the workshop then locate the waving person and answer a question


=== BERT INFERENCE ===
TOKEN	LABEL
[CLS]	O
go	B-VERB
to	O
the	O
workshop	B-LOC
then	O
locate	B-VERB
the	O
waving	B-FILTER
person	B-PERSON
and	O
answer	B-VERB
a	O
question	B-CONTENT
[SEP]	I-FILTER
----------------------

=== SLOTS INPUT FOR T5 ===
intent: go location: workshop | intent: locate filter: waving person: person | intent: answer content: question

=== PLAN GENERATED ===
navigate to workshop | intent -> detect_person with waving at workshop | intent -> answer_question
------------------------------
Select option: 1

=== COMMAND ===
tell me what is the lightest object on the counter


=== BERT INFERENCE ===
TOKEN	LABEL
[CLS]	O
tell	B-VERB
me	I-VERB
what	I-VERB
is	O
the	O
light	B-ATTR
##est	B-ATTR
object	O
on	B-LOC
the	I-LOC
counter	I-LOC
[SEP]	I